In [ ]:
import os
import numpy as np
import imageio
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle as pkl


BASELINE = 12 # cm
PIXEL_LENGTH = 0.0002 # cm
FOCAL_LENGTH = PIXEL_LENGTH*1000 # cm (may need to double check camera model)

l_img_path = './dataset/testing/left_camera/'
r_img_path = './dataset/testing/right_camera/'
disp_dir = './results/'
l_data_path = '../dlc/make_tea_multi-luke-2021-11-15/videos/1636314812_saved_stream.pkl_leftDLC_resnet50_make_tea_multiNov11shuffle1_50000_full.pickle'


def pixelTo3DCameraCoord(left_img, disp_map, coords):
    result_coords = []
    img_dims, disp_dims = left_img.shape, disp_map.shape
    fixed_ratios = [img_dims[dim]/disp_dims[dim] for dim in range(2)]
    for pix in coords:
        # Different ordering of dims between coordinate and images fixed here.
        x_l, y_l = (int(i) for dim, i in enumerate(pix[0]))
        d_x, d_y = int(pix[0][0]/fixed_ratios[1]), int(pix[0][1]/fixed_ratios[0])
        d = im_disp[d_y, d_x]*fixed_ratios[1]
        x_r = int(x_l - d)
        
        # Z is the depth from camera center in cm and X, Y for the other 2 axis.
        Z = BASELINE*FOCAL_LENGTH/(d*PIXEL_LENGTH)
        X, Y = (x_l - img_dims[1]/2)*PIXEL_LENGTH*Z/FOCAL_LENGTH, (y_l - img_dims[0]/2)*PIXEL_LENGTH*Z/FOCAL_LENGTH
        result_coords.append({'left_x':x_l, 'left_y':y_l, 'right_x':x_r, 'X':X, 'Y':Y, 'Z':Z})
    return result_coords

# Getting the coordinate from the dlc folder for now
l_dataf = open(l_data_path, 'rb')
l_data = pkl.load(l_dataf)
l_dataf.close()
left_cam_files = os.listdir(l_img_path)
cam_coordinates = {}

for imgname in l_data.keys():
    filename = '{}.png'.format(imgname)
    if filename in left_cam_files:
        im_l = imageio.imread(l_img_path + filename)
        disp_path = "{}_disp.npy".format(disp_dir + filename.split('.')[0])
        im_disp = np.load(disp_path)
        coordinates = l_data[imgname]['coordinates'][0]
        cam_coordinates[filename] = pixelTo3DCameraCoord(im_l, im_disp, coordinates)
 

In [ ]:
# Select the frame in the dataset folder that will be shown below
shown_frame = 'frame223.png'

lx, ly, rx = [], [], []
for coor in cam_coordinates[shown_frame]:
    lx.append(coor['left_x'])
    ly.append(coor['left_y'])
    rx.append(coor['right_x'])
img_left = imageio.imread(l_img_path + shown_frame)
img_right = imageio.imread(r_img_path + shown_frame)
disp_path = "{}_disp.npy".format(disp_dir + shown_frame.split('.')[0])
img_disparity = np.load(disp_path)
fig = plt.figure(figsize=(16, 24))
fig.add_subplot(3, 1, 1)
plt.imshow(img_right)
plt.scatter(rx, ly)
fig.add_subplot(3, 1, 2)
plt.imshow(img_left)
plt.scatter(lx, ly)
fig.add_subplot(3, 1, 3)
plt.imshow(img_disparity)

In [ ]:
%matplotlib widget

x, y, z = [], [], []
for pix in cam_coordinates[shown_frame]:
    x.append(pix['X'])
    y.append(pix['Y'])
    z.append(pix['Z'])
    
fig = plt.figure()
ax = Axes3D(fig)

# Z-axis increasing the further objects are and X-axis increasing to the right 
# is the original camera view in the left camera coordinate system.
plot_geeks = ax.scatter(x, y, z)
ax.set_title('3D Coordinate Plot')
ax.set_xlabel('x-axis')
ax.set_ylabel('y-axis')
ax.set_zlabel('z-axis')
plt.show()